<a href="https://colab.research.google.com/github/JanMeow/ifc_cases/blob/main/UnderstandingIfcopenshell_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ifcopenshell-colab

Let's start by installing IfcOpenShell. In addition, here are some other libraries that you might find useful. Lark helps parse custom queries, numpy is great for coordinates and matrixes, shapely provides 2D shape analysis, and mathutils provides Vector and Matrix functions.

In [ ]:
!pip install lark
!pip install numpy
!pip install shapely
!pip install mathutils
!pip install ifcopenshell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.4/245.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mathutils: filename=mathutils-3.3.0-cp311-cp311-linux_x86_64.whl size=669889 sha256=2c80b0efe6507a38b6675c6a42d16695d661d56b4416b4c45a832161fc39c2ac
  Stored in directory: /root/.cache/pip/wheels/49/bb/9e/86bd4d162535e81c0f80c64918acf23d920ed0123430d5b4a0
Successfully built mathutils
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 9.5 MB/s eta 0:00:00


Let's import IfcOpenShell to get started.

In [ ]:
import ifcopenshell

We can download a sample model.

In [ ]:
import requests

r = requests.get("https://www.ifcwiki.org/images/e/e3/AC20-FZK-Haus.ifc")
with open("model.ifc", "w") as f:
    f.write(r.text)

Let's get started and analyse our model!

In [ ]:
import ifcopenshell.util
import ifcopenshell.util.element

model = ifcopenshell.open("model.ifc") # Load our IFC model

print(model.schema) # What's the IFC version?
print(len(model.by_type("IfcWall"))) # How many walls are there?

wall = model.by_type("IfcWall")[0] # Grab an arbitrary wall
print(ifcopenshell.util.element.get_psets(wall)) # Get the properties of this particular wall

IFC4
13
{'Pset_WallCommon': {'ThermalTransmittance': 1.5, 'id': 15079}, 'AC_Pset_Name': {'Name': 'Wand-Int-ERDG-4', 'id': 15087}, 'ArchiCADProperties': {'Komplette Element-ID': 'Wand-Int-ERDG-4', 'Kompakte Element-ID': 'Wand-Int-ERDG-4', 'Name des Sachmerkmal-Objekts': '', 'Ursprungsgeschoss': 'Erdgeschoss', 'Raumname': 'Wohnen', 'Raumnummer': '5', 'Ebene': 'Innenwände', 'Typ': 'Wand', 'Geschützt': False, 'Baustoff / Mehrschichtiger Aufbau / Profil / Schraffur': 'Leichtbeton 102890359', 'Etikettentext': '', 'Eindeutige ID': 'BC6F0F70-6195-495E-A2FC-239713029DB1', 'Verknüpfte Änderungen': '', 'Tragende Funktion': 'Nicht definiert', 'Lage': 'Nicht definiert', 'Element-Klassifizierung': 'Wand', 'Umbau-Status': 'Bestand', 'Anzeigen auf Umbau-Filter': 'Alle relevanten Filter', 'Struktur - Typ': 'Einfach', 'ARCHICAD IFC ID': '2XPyKWY018sA1ygZKgQPtU', 'Externe IFC ID': '', 'Baustoff/Mehrschicht/Profil': 'Leichtbeton 102890359', 'Außenseite Oberfläche': 'Anstrich', 'Innenseite Oberfläche': 'An

Typically most ifc components with geometrical properties are inherited from ifc Product

<img src="https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD1/HTML/diagrams/ifcspatialstructureelement.png" style="max-width:80%;" />



In [ ]:
# Iterate through all entities in the IFC file
# all ifc class are subclass of IFC product in IFC 4
for element in model.by_type("IfcProduct")[0:100:5]:
    print(element.is_a()) #print the type of each element
    # print(element.GlobalId) #print the GlobalId of each element

print(len(model.by_type("IfcProduct")))

IfcAnnotation
IfcAnnotation
IfcAnnotation
IfcBeam
IfcDoor
IfcMember
IfcMember
IfcMember
IfcMember
IfcMember
IfcMember
IfcMember
IfcMember
IfcRailing
IfcSlab
IfcWallStandardCase
IfcWallStandardCase
IfcWindow
IfcWindow
IfcWindow
127


In [ ]:
import ifcopenshell.geom
import ifcopenshell.util.shape


settings = ifcopenshell.geom.settings()

for wall in model.by_type("ifcwall")[0:2]:
 shape = ifcopenshell.geom.create_shape(settings, wall)
 shape_matrix = ifcopenshell.util.shape.get_shape_matrix(shape)
 grouped_verts = ifcopenshell.util.shape.get_vertices(shape.geometry)
 print(shape_matrix)
#  print(grouped_verts)

[[1.   0.   0.   7.41]
 [0.   1.   0.   4.25]
 [0.   0.   1.   0.  ]
 [0.   0.   0.   1.  ]]
[[ 0.    1.    0.    7.41]
 [-1.    0.    0.   10.  ]
 [ 0.    0.    1.    0.  ]
 [ 0.    0.    0.    1.  ]]


In [ ]:
settings = ifcopenshell.geom.settings()

for wall in model.by_type("ifcwall")[0:1]:
  shape = ifcopenshell.geom.create_shape(settings, wall)
  grouped_verts  = ifcopenshell.util.shape.get_vertices(shape.geometry)
  print(grouped_verts)
  grouped_edges = ifcopenshell.util.shape.get_edges(shape.geometry)
  print(grouped_edges)

[[ 0.   -0.24  0.  ]
 [ 0.   -0.24  2.5 ]
 [ 4.29 -0.24  2.5 ]
 [ 4.29 -0.24  0.  ]
 [ 4.29  0.    2.5 ]
 [ 4.29  0.    0.  ]
 [ 0.24  0.    2.5 ]
 [ 0.24  0.    0.  ]]
[[0 1]
 [0 3]
 [1 2]
 [2 3]
 [2 4]
 [3 5]
 [4 5]
 [4 6]
 [5 7]
 [6 7]
 [0 7]
 [1 6]]




```
Below is the typical hierachy of an IFC project

IfcProject
 └── IfcSite (Site1)
      ├── IfcBuilding (Building1)
      │    ├── IfcBuildingStorey (Storey1)
      │    └── IfcBuildingStorey (Storey2)
      └── IfcBuilding (Building2)
Here Site has the propery of IsDecomposedBy which will give the childs
One can traverse the tree using .IsDecomposedBy which resolves in a relatonal instance e.g ifcRelAggregates, then one can call again .RelatedObjects for all related childs.


```




<img src="https://standards.buildingsmart.org/IFC/RELEASE/IFC4/ADD1/HTML/figures/ifcspatialstructureelement-spatialstructure.png" style="max-width:60%;" />



In [ ]:
# Iterate through the IFC spatial structure  Project> Site > Building > Storey

def dfs_traverse(base_node, max_depth = 100, depth=0, list_contained_elements = True):

  """
  This function is still in testing ***

  Note that not all child has the property IsDecomposedBy
  The highest level with this property is ifcSite,
  So ideally Site would be the starting note to traverse

  Might need to also add a set to avoid cycle by remembering the nodes visited

  I noticed that as in IfcSpace.IsDecomposedBy, it often returns an empty tuple
  as it is the leaf Node and apprently other building elements are not as a child of Ifc building storey
  but BIM viewer show that it has childs huh?

  Apprently, each level it has ContainElements and IsDecomposedBy
  so as you traverse you need to check if it also has contain elements in that level LOL

  while sometimes it has an empty set as the value for .RelatedObject
  sometimes it has no such attribute in .ContainedElement
  """

  print(f"CURRENT DEPTH : {depth} [TYPE] {base_node.is_a()} [GUID] ({base_node.GlobalId}) [NAME] {base_node.Name}")

  if depth > max_depth:
    print("Max_depth is reached")
    return
  if not hasattr(base_node, "IsDecomposedBy"):
    print("Node does not have attribute IsDecomposedBy")
    return


  # Since the elements e.g columns and wall contained are (surprinsingly) not as part of the spatial hierachy
  if hasattr(base_node, "ContainsElements") and len(base_node.ContainsElements) != 0:
    # for element, one need to use .RelatedElement to iterate through it lol
    for element_rel in base_node.ContainsElements:
      print(f"Number of contained elements: {len(element_rel.RelatedElements)}")

      if list_contained_elements:
        for child_element in element_rel.RelatedElements:
          dfs_traverse(child_element, max_depth, depth = depth, list_contained_elements=list_contained_elements)



  # Turns out IsDecomposedBy depict more of a spatial hierachy
  for child_rel in base_node.IsDecomposedBy:
    print(f"Number of child: {len(child_rel.RelatedObjects)}")
    for child_obj in child_rel.RelatedObjects:
      dfs_traverse(child_obj, max_depth, depth = depth +1, list_contained_elements=list_contained_elements)


In [ ]:
site = model.by_type("ifcsite")[0]

dfs_traverse(site, max_depth =3, depth = 0, list_contained_elements= False)

CURRENT DEPTH : 0 [TYPE] IfcSite [GUID] (0KMpiAlnb52RgQuM1CwVfd) [NAME] Gelaende
Number of child: 1
CURRENT DEPTH : 1 [TYPE] IfcBuilding [GUID] (2hQBAVPOr5VxhS3Jl0O47h) [NAME] FZK-Haus
Number of child: 2
CURRENT DEPTH : 2 [TYPE] IfcBuildingStorey [GUID] (2eyxpyOx95m90jmsXLOuR0) [NAME] Erdgeschoss
Number of contained elements: 38
Number of child: 6
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (347jFE2yX7IhCEIALmupEH) [NAME] 4
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (0e_hbkIQ5DMQlIJ$2V3j_m) [NAME] 3
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (2RSCzLOBz4FAK$_wE8VckM) [NAME] 2
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (0Lt8gR_E9ESeGH5uY_g9e9) [NAME] 5
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (3$f2p7VyLB7eox67SA_zKE) [NAME] 1
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (17JZcMFrf5tOftUTidA0d3) [NAME] 6
CURRENT DEPTH : 2 [TYPE] IfcBuildingStorey [GUID] (273g3wqLzDtfYIl7qqkgcO) [NAME] Dachgeschoss
Number of contained elements: 58
Number of child: 1
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (2dQFggKBb1f

In [ ]:
IfcBuildingStorey = model.by_type("IfcBuildingStorey")[0]
contained_elements = IfcBuildingStorey.ContainsElements
print(contained_elements)
print(contained_elements[0].RelatedElements)
print(IfcBuildingStorey.IsDecomposedBy)

(#14517=IfcRelContainedInSpatialStructure('13J1BKcWxmCqHLM0nJ4nFJ',#12,$,$,(#14502,#15042,#15372,#15848,#16507,#16982,#17040,#17468,#18407,#18465,#18698,#19199,#19504,#20069,#20268,#20299,#20329,#20374,#20598,#20808,#21966,#23024,#23944,#27013,#27421,#27833,#28113,#31079,#31470,#31818,#32098,#32407,#32829,#33109,#33389,#33672,#34509,#35053),#479),)
(#14502=IfcStair('38a9vdh9bF5Qg28GWyHhlr',#12,'Wendeltreppe',$,$,#502,#14498,'79A67A01-C95B-4209-86-9A-74983B65305C',$), #15042=IfcWallStandardCase('2XPyKWY018sA1ygZKgQPtU',#12,'Wand-Int-ERDG-4',$,$,#14983,#15037,'BC6F0F70-6195-495E-A2-FC-239713029DB1',$), #15372=IfcAnnotation('2TSghi3E94BuNE_F6jBcWe',#12,$,$,$,#15264,#15369), #15848=IfcAnnotation('0j7WClqLTB0gkHE605BLnY',#12,$,$,$,#15384,#15845), #16507=IfcAnnotation('3BTqRqk7D5EhoEfLy$YPgs',#12,$,$,$,#15859,#16504), #16982=IfcAnnotation('11pyNSTMP9zfUCy679gsvT',#12,$,$,$,#16518,#16979), #17040=IfcWallStandardCase('3PfS__Y_DBAfq5naM6zD2Z',#12,'Wand-Int-ERDG-2',$,$,#16993,#17035,'40F78310-9E

In [ ]:
# Iterate through the IFC spatial structure  Project> Site > Building > Storey
from collections import deque


def bfs_traverse(base_node, list_contained_elements = True):


  queue = deque([base_node])
  depth = 0

  while len(queue) !=0 :

    current_node = queue.popleft()
    print(f"CURRENT DEPTH : {depth} [TYPE] {current_node.is_a()} [GUID] ({current_node.GlobalId}) [NAME] {current_node.Name}")


    if hasattr(current_node, "ContainsElements") and len(current_node.ContainsElements) != 0:
      for element_rel in current_node.ContainsElements:
        print(f"Contained Elements: {len(element_rel.RelatedElements)}")
        if list_contained_elements:
          for child_element in element_rel.RelatedElements:
            queue.append(child_element)

    if hasattr(current_node, "IsDecomposedBy") and len(current_node.IsDecomposedBy) != 0:
      depth +=1
      for child_rel in current_node.IsDecomposedBy:
        print(f"Number of child: {len(child_rel.RelatedObjects)}")
        for child_obj in child_rel.RelatedObjects:
          queue.append(child_obj)



  print("Function ended, No more spatial child")



In [ ]:
site = model.by_type("ifcproject")[0]
bfs_traverse(site, list_contained_elements = False)

CURRENT DEPTH : 0 [TYPE] IfcProject [GUID] (0lY6P5Ur90TAQnnnI6wtnb) [NAME] Projekt-FZK-Haus
Number of child: 1
CURRENT DEPTH : 1 [TYPE] IfcSite [GUID] (0KMpiAlnb52RgQuM1CwVfd) [NAME] Gelaende
Number of child: 1
CURRENT DEPTH : 2 [TYPE] IfcBuilding [GUID] (2hQBAVPOr5VxhS3Jl0O47h) [NAME] FZK-Haus
Number of child: 2
CURRENT DEPTH : 3 [TYPE] IfcBuildingStorey [GUID] (2eyxpyOx95m90jmsXLOuR0) [NAME] Erdgeschoss
Contained Elements: 38
Number of child: 6
CURRENT DEPTH : 4 [TYPE] IfcBuildingStorey [GUID] (273g3wqLzDtfYIl7qqkgcO) [NAME] Dachgeschoss
Contained Elements: 58
Number of child: 1
CURRENT DEPTH : 5 [TYPE] IfcSpace [GUID] (347jFE2yX7IhCEIALmupEH) [NAME] 4
CURRENT DEPTH : 5 [TYPE] IfcSpace [GUID] (0e_hbkIQ5DMQlIJ$2V3j_m) [NAME] 3
CURRENT DEPTH : 5 [TYPE] IfcSpace [GUID] (2RSCzLOBz4FAK$_wE8VckM) [NAME] 2
CURRENT DEPTH : 5 [TYPE] IfcSpace [GUID] (0Lt8gR_E9ESeGH5uY_g9e9) [NAME] 5
CURRENT DEPTH : 5 [TYPE] IfcSpace [GUID] (3$f2p7VyLB7eox67SA_zKE) [NAME] 1
CURRENT DEPTH : 5 [TYPE] IfcSpace [GU

In [ ]:
# Now we have constructed a way to travese through the ifc4 tree
# We might want to also get all the property of the certain node


# Accessing all entities without filtering as some entities are not inheritance of ifc root
all_entities = list(model)
print(len(all_entities))
root = model.by_type("ifcproject")[0]
wall = model.by_type("ifcwall")[0]

44249


In [ ]:
def traverse_upward(entity):
  parents = model.get_inverse(entity)

  # Get spatial Relation
  if not parents:
    return None

  for parent in parents:
    if parent.is_a() == "IfcRelAggregates":
      print(parent)
      return parent

    if parent.is_a() == "IfcRelContainedInSpatialStructure":
      print(parent)
      return parent


wall = model.by_type("ifcwall")[0]
traverse_upward(wall)

#14517=IfcRelContainedInSpatialStructure('13J1BKcWxmCqHLM0nJ4nFJ',#12,$,$,(#14502,#15042,#15372,#15848,#16507,#16982,#17040,#17468,#18407,#18465,#18698,#19199,#19504,#20069,#20268,#20299,#20329,#20374,#20598,#20808,#21966,#23024,#23944,#27013,#27421,#27833,#28113,#31079,#31470,#31818,#32098,#32407,#32829,#33109,#33389,#33672,#34509,#35053),#479)


#14517=IfcRelContainedInSpatialStructure('13J1BKcWxmCqHLM0nJ4nFJ',#12,$,$,(#14502,#15042,#15372,#15848,#16507,#16982,#17040,#17468,#18407,#18465,#18698,#19199,#19504,#20069,#20268,#20299,#20329,#20374,#20598,#20808,#21966,#23024,#23944,#27013,#27421,#27833,#28113,#31079,#31470,#31818,#32098,#32407,#32829,#33109,#33389,#33672,#34509,#35053),#479)

In [ ]:
# Originally, I thought I can traverse and write it into .ifc but I was too nairve
# Apparently, BIM viwer can only view it when it has a basic file structure like Project>Site>....
# According to chatGPT If  IFC file  contains geometry that's already tied to entities (like IfcWall or IfcSlab), the absence of a context doesn't prevent visualization.
# context not mandatory for a valid IFC file but is recommended  to define how geometry is represented (e.g., 3D models, plans, etc.).

def create_basic_project_strcuture(entity, schema = "IFC4", relationship = "IfcRelAggregates"):

  spatial_hierachy = ["IfcProject","IfcSite", "IfcBuilding", "IfcBuildingStorey"]

  # Instantiate a new model
  new_model = ifcopenshell.file(schema = schema)

  # Get the spatial level of the entity you want to create
  spatial_level = entity.is_a()

  # Create_basic_skeleton
  def create():
    prev = None
    for item in spatial_hierachy:

      if item == "IfcProject":
        new_spatial_layer = new_model.create_entity(item, GlobalId=ifcopenshell.guid.new(), Name= "Sample" + item)
        context = new_model.create_entity("IfcGeometricRepresentationContext", ContextType="Model", ContextIdentifier="Building Model")
        new_spatial_layer.RepresentationContexts = [context]
      else:
        new_spatial_layer = new_model.create_entity(item, GlobalId=ifcopenshell.guid.new(), Name= "Sample" + item)

      if prev != None:
        new_relationship = new_model.create_entity(relationship, GlobalId=ifcopenshell.guid.new(), RelatingObject=prev, RelatedObjects=[new_spatial_layer])

      prev = new_spatial_layer
    return new_model, prev

  return create()

In [ ]:
from typing_extensions import Annotated
def create_ifc_for_partial_model(entity, schema = "IFC4", write_file = True, file_path = "new_model.ifc", save_props = False):


  # Create_basic_skeleton
  new_model, storey = create_basic_project_strcuture(entity, schema = "IFC4", relationship = "IfcRelAggregates")

  partial_grapth = model.traverse(entity)


  # Copying the entities into an object and linking it to the storey level
  for i,node in enumerate(partial_grapth):
    copied_entity = new_model.add(node)
    if i == 0:
      root_node = copied_entity

    # Traverse does not contain the custom property, so we need to get those also
    if save_props and hasattr(node, "IsDefinedBy"):
      props = node.IsDefinedBy
      for prop in props:
        new_model.add(prop)

  # Linking to the storey level
  new_model.create_entity(
    "IfcRelContainedInSpatialStructure",
    GlobalId=ifcopenshell.guid.new(),
    RelatingStructure=storey,
    RelatedElements=[root_node],
)
  if write_file:
    new_model.write(file_path)

  return new_model

model = ifcopenshell.open("model.ifc")
wall = model.by_type("ifcwall")[0]

new_model = create_ifc_for_partial_model(wall)


# Lets try traverse through the new model
site = new_model.by_type("ifcsite")[0]
bfs_traverse(site, max_depth =3, list_contained_elements = True)

CURRENT DEPTH : 0 [TYPE] IfcSite [GUID] (1lZu33ynXCyOEErokZetl7) [NAME] SampleIfcSite
Number of child: 1
CURRENT DEPTH : 1 [TYPE] IfcBuilding [GUID] (202_ASN1v8neP$s2CAxmce) [NAME] SampleIfcBuilding
Number of child: 1
CURRENT DEPTH : 2 [TYPE] IfcBuildingStorey [GUID] (1EdZ4hsa5EjxHpmoK91Pm8) [NAME] SampleIfcBuildingStorey
Contained Elements: 1
CURRENT DEPTH : 2 [TYPE] IfcWallStandardCase [GUID] (2XPyKWY018sA1ygZKgQPtU) [NAME] Wand-Int-ERDG-4
Function ended, No more spatial child


In [ ]:
# Check the attributes of an entity
current_node = model.by_type("ifcwall")[0]

# IsDefinedBy gives you
print(current_node.IsDefinedBy)


for rel in current_node.IsDefinedBy:
  print(rel)

# List all attributes
# current_node.__dir__()

(#15082=IfcRelDefinesByProperties('1XHlCNqAFH0tlvMzR6N0fY',#12,$,$,(#15042),#15079), #15090=IfcRelDefinesByProperties('28j1CHGQqsg8bOIv_AhSyO',#12,$,$,(#15042),#15087), #15157=IfcRelDefinesByProperties('16oiHR3wk3Xm_oKuvXa7iw',#12,$,$,(#15042),#15124), #15171=IfcRelDefinesByProperties('2rF$g0UZGGPpjaTWxrJdtz',#12,$,$,(#15042),#15169), #15231=IfcRelDefinesByProperties('3Q0nMR5elnJFWzAhgkZqe1',#12,$,$,(#15042),#15229))
#15082=IfcRelDefinesByProperties('1XHlCNqAFH0tlvMzR6N0fY',#12,$,$,(#15042),#15079)
#15090=IfcRelDefinesByProperties('28j1CHGQqsg8bOIv_AhSyO',#12,$,$,(#15042),#15087)
#15157=IfcRelDefinesByProperties('16oiHR3wk3Xm_oKuvXa7iw',#12,$,$,(#15042),#15124)
#15171=IfcRelDefinesByProperties('2rF$g0UZGGPpjaTWxrJdtz',#12,$,$,(#15042),#15169)
#15231=IfcRelDefinesByProperties('3Q0nMR5elnJFWzAhgkZqe1',#12,$,$,(#15042),#15229)


So we have made a function to export individual building element.
However, as we use the traverse function, we do not get the property from other relationship
Lets back to the main goal of this notebook, which is to translate ifc4 into ifc5


In [ ]:
# Create a script to also add custom propety
# Here propety value has to be of an ifc type, for example IFCREAL(304.00) or IFC Integer

def set_property (model, entity, property_name,property_value, property_type = "IFCPROPERTYSINGLEVALUE"):

  #Here propety value has to be of an ifc type, for example IFCREAL(304.00) or IFC Integer
  # For type safty in the future, might need to add conversion
  NominalValue =real_value = model.create_entity("IfcReal", property_value)

  # Create the Value object and the property object that links the two
  new_property = model.create_entity(property_type, Name = property_name, NominalValue = NominalValue )
  property_set = model.create_entity("IfcPropertySet", GlobalId=ifcopenshell.guid.new(), Name=f"{property_name}_set", HasProperties = [new_property])

  # Create Relational Instance
  model.create_entity(
        "IfcRelDefinesByProperties",
        GlobalId=f"NEW_REL_{ifcopenshell.guid.new()}",
        RelatingPropertyDefinition=property_set,
        RelatedObjects=[entity]
    )
  return entity


wall = model.by_type("ifcwall")[0]


# Set Propety
wall = set_property(model, wall, property_name = "hahaha", property_value = 500)

# Check if the NEW_REL relationship is created
# for rel in wall.IsDefinedBy:
#   print(rel.GlobalId)


#try travesing the wall
graph = model.traverse(wall)

# Lets get the model and see if it has such property as sometimes in BIM viewers such property are not here
create_ifc_for_partial_model(wall, save_props = True)

# IFC4 to IFC5 Converter

GREAT ! So we have succefully exported ifc elements from a larger file but also save all the previous properties !
Lets go back to our main quest, which is translating ifc4 to ifc5 !!!

In [ ]:
# First lets set up and revise some basic class prims for ifc5
export = [
    {
        "disclaimaer":"MeowMeowIFC"
    }
]

ifc5_type = ["UsdGeom:Mesh","UsdGeom:Xform", "UsdGeom:Mesh","UsdGeom:BasisCurves", "UsdShade:Material", "UsdShade:Shader"]

# IFC5 uses a proxy later as UsdGeom:Xform
ifc5_class = {

  "def": "class",
  "type": "",
  "name": "",
  "children":[],
}

ifc5_over = {

  "def": "over",
  "name": "",
  "attributes":{}
}

ifc5_def = {

  "def": "def",
  "type":"",
  "name": "",
  "inherits":[]
}

# Lets start from the simple IFC file we got from the new_model.
test_model = ifcopenshell.open("new_model.ifc")

We begin traversing the model but we would like to create a function so that as we traverse, we also create an USD/ifc5 object
and at each node we create the graph structure as we traverse and then push it into a result :)

A lot of conversion are not known yet in ifc5,
so I at the end decided not to translate all of the property for over as I dont know what is the name tag for those attributes.
But I want to at least translate the geometry

In [ ]:
# Use ifc geom util to create shape
# Getting Geometric information is much easier using.

settings = ifcopenshell.geom.settings()

shape = ifcopenshell.geom.create_shape(settings, wall)

# points are local coordinate
points  = ifcopenshell.util.shape.get_vertices(shape.geometry)
grouped_edges = ifcopenshell.util.shape.get_edges(shape.geometry)


# Some information are useful in ifc5
T_matrix = ifcopenshell.util.shape.get_shape_matrix(shape)
face_vertex_index = shape.geometry.faces

# Note that global coordinate could be dervied from grouped_verts.hstack(1) XT_matrix Tranpose

print(T_matrix)
print(grouped_verts)
print(face_vertex_index)

[[1.   0.   0.   7.41]
 [0.   1.   0.   4.25]
 [0.   0.   1.   0.  ]
 [0.   0.   0.   1.  ]]
[[ 0.   -0.24  0.  ]
 [ 0.   -0.24  2.5 ]
 [ 4.29 -0.24  2.5 ]
 [ 4.29 -0.24  0.  ]
 [ 4.29  0.    2.5 ]
 [ 4.29  0.    0.  ]
 [ 0.24  0.    2.5 ]
 [ 0.24  0.    0.  ]]
(1, 0, 3, 2, 1, 3, 2, 3, 5, 4, 2, 5, 4, 5, 7, 6, 4, 7, 6, 7, 0, 1, 6, 0, 0, 7, 3, 3, 7, 5, 2, 6, 1, 4, 6, 2)


In [ ]:
import numpy as np
# Global Coordinate just for fun
ones = np.ones(shape = (grouped_verts.shape[0],1))
stacked = np.hstack((grouped_verts, ones))

global_coor = stacked@ T_matrix.T
global_coor

# You just need to drop the last column if you want
global_coor = global_coor[:,0:-1]

global_coor

array([[ 7.41,  4.01,  0.  ],
       [ 7.41,  4.01,  2.5 ],
       [11.7 ,  4.01,  2.5 ],
       [11.7 ,  4.01,  0.  ],
       [11.7 ,  4.25,  2.5 ],
       [11.7 ,  4.25,  0.  ],
       [ 7.65,  4.25,  2.5 ],
       [ 7.65,  4.25,  0.  ]])

In [ ]:
# Lets turn the geometry information into a function
def get_geometry_info(entity):
  settings = ifcopenshell.geom.settings()
  shape = ifcopenshell.geom.create_shape(settings, entity)
  points  = ifcopenshell.util.shape.get_vertices(shape.geometry)
  T_matrix = ifcopenshell.util.shape.get_shape_matrix(shape)
  faceVertexIndices = list(shape.geometry.faces)
  return T_matrix, points, faceVertexIndices

In [ ]:
get_geometry_info(wall)

(array([[1.  , 0.  , 0.  , 7.41],
        [0.  , 1.  , 0.  , 4.25],
        [0.  , 0.  , 1.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  ]]),
 array([[ 0.  , -0.24,  0.  ],
        [ 0.  , -0.24,  2.5 ],
        [ 4.29, -0.24,  2.5 ],
        [ 4.29, -0.24,  0.  ],
        [ 4.29,  0.  ,  2.5 ],
        [ 4.29,  0.  ,  0.  ],
        [ 0.24,  0.  ,  2.5 ],
        [ 0.24,  0.  ,  0.  ]]),
 [1,
  0,
  3,
  2,
  1,
  3,
  2,
  3,
  5,
  4,
  2,
  5,
  4,
  5,
  7,
  6,
  4,
  7,
  6,
  7,
  0,
  1,
  6,
  0,
  0,
  7,
  3,
  3,
  7,
  5,
  2,
  6,
  1,
  4,
  6,
  2])

In [ ]:
wall = model.by_type("ifcwall")[0]
site = new_model.by_type("ifcsite")[0]
print(wall.IsDefinedBy)

(#15082=IfcRelDefinesByProperties('1XHlCNqAFH0tlvMzR6N0fY',#12,$,$,(#15042),#15079), #15090=IfcRelDefinesByProperties('28j1CHGQqsg8bOIv_AhSyO',#12,$,$,(#15042),#15087), #15157=IfcRelDefinesByProperties('16oiHR3wk3Xm_oKuvXa7iw',#12,$,$,(#15042),#15124), #15171=IfcRelDefinesByProperties('2rF$g0UZGGPpjaTWxrJdtz',#12,$,$,(#15042),#15169), #15231=IfcRelDefinesByProperties('3Q0nMR5elnJFWzAhgkZqe1',#12,$,$,(#15042),#15229), #79110=IfcRelDefinesByProperties('NEW_REL_1Zs1lK3Y9EpwZ_RkccJLb$',$,$,$,(#15042),#79109))


In [ ]:
for rep in wall.Representation.Representations:
  print(rep)

# print( wall.Representation.Representations[0].Items[0].Position.Location)
# print( wall.Representation.Representations[0].Items[0].Position.Axis)
# print( wall.Representation.Representations[0].Items[0].Position.RefDirection)

Axis = [wall.Representation.Representations[0].Items[0].Position.Location.Coordinates,
wall.Representation.Representations[0].Items[0].Position.Location.Coordinates]

Axis

#15016=IfcShapeRepresentation(#118,'Body','SweptSolid',(#15006))
#15024=IfcShapeRepresentation(#375,'Box','BoundingBox',(#15023))
#15033=IfcShapeRepresentation(#15026,'Axis','Curve2D',(#15031))


[(0.0, 0.0, 0.0), (0.0, 0.0, 0.0)]

In [ ]:
# We begin traversing the model but we would like to create a function so that as we traverse, we also create an USD/ifc5 object
# and at each node we create the graph structure as we traverse and then push it into a result :)
# I will also break down the functon into subfunction in the future

# 1. Lets modify the previous bfs function, alternatively we could also modify the dfs but i want to keep a bit of a hierarchical structure so i chose BFS

def build_graph(node, prev= None, get_props = True):

  # For each node, ifc5 has a proxy layer to wrap around it so lets also create it first!
  # Here children is more in a sptial way while inherits is more in programming way for inheriting characterisitcs
  # For example, in case of project, the children will be site, then building, then storey etc....


  ifc5_current_class = {
  "def": "class",
  "type": "UsdGeom:Xform",
  "name": f"MEOW_{ifcopenshell.guid.new()}",
  "children":[],
  }

  # Turn current node into an JSON/ USD Object first
  # Inherits of def object is also put in </>.
  ifc5_current_def = {
    "def": "def",
    "type":"UsdGeom:Xform",
    "name": node.Name,
    "inherits":[f"</{ifc5_current_class['name']}>"]
    }

  if prev:
    prev["children"].append(ifc5_current_def)
    print(prev)

  # Create Overlaying layers for adding information
  # over itself takes on the same name as the class it is overlaied to
  # I noticed that here the classname for wall is IfcWallStandardCase but its changed to wall so it might need extra conversion
  # Lets figure out how to solve this later la sigh so stupid
  # THe first over is a inheritance from the standard ifc object type propoerties
  ifc5_over = {
  "def": "over",
  "name": ifc5_current_class['name'],
  "attributes":{
      "ifc5:class":{
          "code":node.is_a(),
          "uri":f"https://identifier.buildingsmart.org/uri/buildingsmart/ifc/4.3/class/{node.is_a()}"
      }
    }
  }

  export = [ifc5_current_class, ifc5_current_def, ifc5_over]

  # Get all the props for other overlaying layers
  # this part is not fully done because I see that some attibutes in fact change their names
  # but i have no idea where to find new names and the conversion :(
  # So lets use the old attribute for now
  if get_props and hasattr(node, "IsDefinedBy"):
    for prop in node.IsDefinedBy:
      prop_over = {
        "def": "over",
        "name": ifc5_current_class['name'],
        "attributes":{}
      }
      export.append(prop_over)

  # Get Geometric Information
  if hasattr(node, "Representation"):
    if node.Representation != None:
      T_matrix, points, faceVertexIndices = get_geometry_info(node)
      conveter = {
          "Body":{
              "name":"UsdGeom:Mesh",
              "attributes":{
                  "faceVertexIndices":faceVertexIndices,
                  "points":points.tolist()
              }
          }
      }

      # Base Overlay for coordinate
      ifc_geo_base_over = {
          "def": "over",
          "name": ifc5_current_class['name'],
          "attributes":{
            "xformOp": {
            "transform": T_matrix.tolist()
          }
        }
      }

      export.append(ifc_geo_base_over)


      # No need to translate all of them ? like Box you dont need to translate
      # For wall they break it down to multiple sublayers
      # for rep in node.Representation.Representations:
      #   if rep.RepresentationIdentifier in conveter:
      #     ifc5_geo_def = {
      #       "def": "def",
      #       "type":conveter[rep.RepresentationIdentifier]["name"],
      #       "name": rep.RepresentationIdentifier,
      #       "inherits":[f"</{ifc5_current_class['name']}_{rep.RepresentationIdentifier}>"]
      #     }
      #     ifc5_geo_class = {
      #         "def": "class",
      #         "type": conveter[rep.RepresentationIdentifier]["name"],
      #         "name": f"{ifc5_current_class['name']}_{rep.RepresentationIdentifier}",
      #     }
      #     ifc5_geo_over = {
      #         "def": "over",
      #         "name": f"{ifc5_current_class['name']}_{rep.RepresentationIdentifier}",
      #         "attributes":conveter[rep.RepresentationIdentifier]["attributes"]
      #     }
      #     export.extend([ifc5_geo_def, ifc5_geo_class, ifc5_geo_over])

  return export




In [ ]:
from collections import deque


def bfs_traverse_build(base_node, list_contained_elements = True):

  queue = deque([base_node])
  depth = 0
  prev = None
  out_Js = []

  while len(queue) !=0 :

    current_node = queue.popleft()
    print(f"CURRENT DEPTH : {depth} [TYPE] {current_node.is_a()} [GUID] ({current_node.GlobalId}) [NAME] {current_node.Name}")

    export = build_graph(current_node, prev= prev, get_props = True)
    prev = export[0]

    if hasattr(current_node, "ContainsElements") and len(current_node.ContainsElements) != 0:
      for element_rel in current_node.ContainsElements:
        print(f"Contained Elements: {len(element_rel.RelatedElements)}")
        if list_contained_elements:
          for child_element in element_rel.RelatedElements:
            queue.append(child_element)

    if hasattr(current_node, "IsDecomposedBy") and len(current_node.IsDecomposedBy) != 0:
      depth +=1
      for child_rel in current_node.IsDecomposedBy:
        print(f"Number of child: {len(child_rel.RelatedObjects)}")
        for child_obj in child_rel.RelatedObjects:
          queue.append(child_obj)

    out_Js += export


  print("Function ended, No more spatial child")

  return out_Js


base_node = new_model.by_type("ifcproject")[0]
out_Js = bfs_traverse_build(base_node, list_contained_elements = True)

CURRENT DEPTH : 0 [TYPE] IfcProject [GUID] (1FODwMAQT3awaXWU8edTv4) [NAME] SampleIfcProject
Number of child: 1
CURRENT DEPTH : 1 [TYPE] IfcSite [GUID] (1lZu33ynXCyOEErokZetl7) [NAME] SampleIfcSite
{'def': 'class', 'type': 'UsdGeom:Xform', 'name': 'MEOW_1OPPevFBn1kQuxFd$UWocL', 'children': [{'def': 'def', 'type': 'UsdGeom:Xform', 'name': 'SampleIfcSite', 'inherits': ['</MEOW_2OfvBL0OL3EfHzDCCEkZM$>']}]}
Number of child: 1
CURRENT DEPTH : 2 [TYPE] IfcBuilding [GUID] (202_ASN1v8neP$s2CAxmce) [NAME] SampleIfcBuilding
{'def': 'class', 'type': 'UsdGeom:Xform', 'name': 'MEOW_2OfvBL0OL3EfHzDCCEkZM$', 'children': [{'def': 'def', 'type': 'UsdGeom:Xform', 'name': 'SampleIfcBuilding', 'inherits': ['</MEOW_1kjjQPVy1DnQ$2nDmCZdQA>']}]}
Number of child: 1
CURRENT DEPTH : 3 [TYPE] IfcBuildingStorey [GUID] (1EdZ4hsa5EjxHpmoK91Pm8) [NAME] SampleIfcBuildingStorey
{'def': 'class', 'type': 'UsdGeom:Xform', 'name': 'MEOW_1kjjQPVy1DnQ$2nDmCZdQA', 'children': [{'def': 'def', 'type': 'UsdGeom:Xform', 'name': '

In [ ]:
import json

def export_to_json(data, file_path):
    """
    Export a list of dictionaries to a JSON file.

    Args:
        data (list): List of dictionaries to export.
        file_path (str): Path to the JSON file to save.

    Returns:
        None
    """
    try:
        with open(file_path, 'w') as json_file:
            json.dump(data, json_file, indent=2,separators=(',', ':'))
        print(f"JSON file successfully saved to {file_path}")
    except Exception as e:
        print(f"An error occurred while exporting JSON: {e}")


export_to_json(out_Js, "output.json")


JSON file successfully saved to output.json


# Turning BIM data into Graph

I notice that while I translated the file to ifc5 format, the online ifc5 viewer is unable to view it and has some weird iterations on the file.
Unsure how to furhter better my code, I decide to work on a new project which is to translate different components in to spatial graph
aka to ask the queston:
what are connected to the current building element I am at ?

In [ ]:
class SpatialGraph:
  def __init__(self):
      self.nodes = []
      self.sizes = 0


class Nodes:
  def __init(self):
    self.connected = []
    return



In [ ]:
# Check if object are touching each other


# Iterate through the IFC spatial structure  Project> Site > Building > Storey
from collections import deque


def bfs_traverse(base_node, list_contained_elements = True):


  queue = deque([base_node])
  depth = 0

  while len(queue) !=0 :

    current_node = queue.popleft()
    print(f"CURRENT DEPTH : {depth} [TYPE] {current_node.is_a()} [GUID] ({current_node.GlobalId}) [NAME] {current_node.Name}")


    if hasattr(current_node, "ContainsElements") and len(current_node.ContainsElements) != 0:
      for element_rel in current_node.ContainsElements:
        print(f"Contained Elements: {len(element_rel.RelatedElements)}")
        if list_contained_elements:
          for child_element in element_rel.RelatedElements:
            queue.append(child_element)

    if hasattr(current_node, "IsDecomposedBy") and len(current_node.IsDecomposedBy) != 0:
      depth +=1
      for child_rel in current_node.IsDecomposedBy:
        print(f"Number of child: {len(child_rel.RelatedObjects)}")
        for child_obj in child_rel.RelatedObjects:
          queue.append(child_obj)



  print("Function ended, No more spatial child")

site = model.by_type("IfcBuilding")[0]

bfs_traverse(site, list_contained_elements = False)

CURRENT DEPTH : 0 [TYPE] IfcBuilding [GUID] (2hQBAVPOr5VxhS3Jl0O47h) [NAME] FZK-Haus
Number of child: 2
CURRENT DEPTH : 1 [TYPE] IfcBuildingStorey [GUID] (2eyxpyOx95m90jmsXLOuR0) [NAME] Erdgeschoss
Contained Elements: 38
Number of child: 6
CURRENT DEPTH : 2 [TYPE] IfcBuildingStorey [GUID] (273g3wqLzDtfYIl7qqkgcO) [NAME] Dachgeschoss
Contained Elements: 58
Number of child: 1
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (347jFE2yX7IhCEIALmupEH) [NAME] 4
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (0e_hbkIQ5DMQlIJ$2V3j_m) [NAME] 3
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (2RSCzLOBz4FAK$_wE8VckM) [NAME] 2
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (0Lt8gR_E9ESeGH5uY_g9e9) [NAME] 5
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (3$f2p7VyLB7eox67SA_zKE) [NAME] 1
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (17JZcMFrf5tOftUTidA0d3) [NAME] 6
CURRENT DEPTH : 3 [TYPE] IfcSpace [GUID] (2dQFggKBb1fOc1CqZDIDlx) [NAME] 7
Function ended, No more spatial child


In [ ]:
wall1 = model.by_type("ifcwall")[0]
wall2 = model.by_type("ifcwall")[1]

In [ ]:
# check if two volumes are touching => share vertex / face => meaning one of the face of ObjA and objB are linearly dependent

# Lets turn the geometry information into a function
def test_intersection(entity):
  settings = ifcopenshell.geom.settings()
  shape = ifcopenshell.geom.create_shape(settings, entity)
  faces  = ifcopenshell.util.shape.get_faces(shape.geometry)
  T_matrix = ifcopenshell.util.shape.get_shape_matrix(shape)
  faceVertexIndices = list(shape.geometry.faces)
  return faces

test_intersection(wall1)

array([[1, 0, 3],
       [2, 1, 3],
       [2, 3, 5],
       [4, 2, 5],
       [4, 5, 7],
       [6, 4, 7],
       [6, 7, 0],
       [1, 6, 0],
       [0, 7, 3],
       [3, 7, 5],
       [2, 6, 1],
       [4, 6, 2]], dtype=int32)

In [ ]:
test_intersection(wall2)

array([[ 2,  0,  7],
       [ 7,  6,  3],
       [ 2,  7,  3],
       [ 7,  0,  1],
       [ 6,  5,  4],
       [ 3,  6,  4],
       [ 0,  8,  9],
       [ 1,  0,  9],
       [10,  7,  1],
       [ 9, 10,  1],
       [11,  6,  7],
       [10, 11,  7],
       [12,  5,  6],
       [11, 12,  6],
       [ 5, 12, 13],
       [ 5, 13,  4],
       [ 3,  4, 13],
       [14,  3, 13],
       [14,  2,  3],
       [14, 15,  2],
       [15,  8,  0],
       [ 2, 15,  0],
       [14, 11, 10],
       [11, 14, 13],
       [11, 13, 12],
       [14, 10, 15],
       [10,  9,  8],
       [15, 10,  8]], dtype=int32)

ifc js =Y

limitaton of ifc js ?
ifc js component
comminication =json

communicte warning from back to front
bottleneck is ifc is strcutured and have to use that structure to do the check

different check rule
in the orchestrator